#Imports

In [2]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.1 MB/s eta 0:00:00


In [3]:
import itertools
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForMaskedLM
import time
from itertools import groupby
import re
from functools import reduce
from collections import defaultdict
from jiwer import wer, cer
from nltk.translate.bleu_score import corpus_bleu

#Rule-based OOV handler

In [4]:

nVowels =26
# 46
consonants = []
consonantsUni = []

# 26
vowels = []
vowelsUni = []
vowelModifiersUni = []

# 6
specialConsonants = []
specialConsonantsUni = []
# 2
specialCharUni = []
specialChar = []


def initializeVar():

    vowelsUni.append("ඌ")
    vowels.append("oo")
    vowelModifiersUni.append("ූ")

    vowelsUni.append("ඕ")
    vowels.append("o\\)")
    vowelModifiersUni.append("ෝ")
    vowelsUni.append("ඕ")
    vowels.append("oe")
    vowelModifiersUni.append("ෝ")
    vowelsUni.append("ආ")
    vowels.append("aa")
    vowelModifiersUni.append("ා")
    vowelsUni.append("ආ")
    vowels.append("a\\)")
    vowelModifiersUni.append("ා")
    vowelsUni.append("ඈ")
    vowels.append("Aa")
    vowelModifiersUni.append("ෑ")
    vowelsUni.append("ඈ")
    vowels.append("A\\)")
    vowelModifiersUni.append("ෑ")
    vowelsUni.append("ඈ")
    vowels.append("ae")
    vowelModifiersUni.append("ෑ")
    vowelsUni.append("ඊ")
    vowels.append("ii")
    vowelModifiersUni.append("ී")
    vowelsUni.append("ඊ")
    vowels.append("i\\)")
    vowelModifiersUni.append("ී")
    vowelsUni.append("ඊ")
    vowels.append("ie")
    vowelModifiersUni.append("ී")
    vowelsUni.append("ඊ")
    vowels.append("ee")
    vowelModifiersUni.append("ී")
    vowelsUni.append("ඒ")
    vowels.append("ea")
    vowelModifiersUni.append("ේ")
    vowelsUni.append("ඒ")
    vowels.append("e\\)")
    vowelModifiersUni.append("ේ")
    vowelsUni.append("ඒ")
    vowels.append("ei")
    vowelModifiersUni.append("ේ")
    vowelsUni.append("ඌ")
    vowels.append("uu")
    vowelModifiersUni.append("ූ")
    vowelsUni.append("ඌ")
    vowels.append("u\\)")
    vowelModifiersUni.append("ූ")

    vowelsUni.append("ඖ")
    vowels.append("au")
    vowelModifiersUni.append("ෞ")

    vowelsUni.append("ඇ")
    vowels.append("\\a")
    vowelModifiersUni.append("ැ")

    vowelsUni.append("අ")
    vowels.append("a")
    vowelModifiersUni.append("")

    vowelsUni.append("ඇ")
    vowels.append("A")
    vowelModifiersUni.append("ැ")
    vowelsUni.append("ඉ")
    vowels.append("i")
    vowelModifiersUni.append("ි")
    vowelsUni.append("එ")
    vowels.append("e")
    vowelModifiersUni.append("ෙ")
    vowelsUni.append("උ")
    vowels.append("u")
    vowelModifiersUni.append("ු")
    vowelsUni.append("ඔ")
    vowels.append("o")
    vowelModifiersUni.append("ො")
    vowelsUni.append("ඓ")
    vowels.append("I")
    vowelModifiersUni.append("ෛ")

    specialConsonantsUni.append("ං")
    specialConsonants.append("\\n")

    specialConsonantsUni.append("ඃ")
    specialConsonants.append("\\h")
    specialConsonantsUni.append("ඞ")
    specialConsonants.append("\\N")
    specialConsonantsUni.append("ඍ")
    specialConsonants.append("\\R")
    # special characher Repaya
    specialConsonantsUni.append("ර්"+"\u200D")
    specialConsonants.append("R")
    specialConsonantsUni.append("ර්"+"\u200D")
    specialConsonants.append("\\r")

    consonantsUni.append("ඬ")
    consonants.append("nnd")

    consonantsUni.append("ඳ")
    consonants.append("nndh")

    consonantsUni.append("ඟ")
    consonants.append("nng")

    consonantsUni.append("ත")
    consonants.append("th")

    consonantsUni.append("ධ")
    consonants.append("dh")
    consonantsUni.append("ඝ")
    consonants.append("gh")
    consonantsUni.append("ච")
    consonants.append("ch")
    consonantsUni.append("ඵ")
    consonants.append("ph")
    consonantsUni.append("භ")
    consonants.append("bh")
    consonantsUni.append("ඣ")
    consonants.append("jh")
    consonantsUni.append("ෂ")
    consonants.append("sh")
    consonantsUni.append("ඥ")
    consonants.append("GN")
    consonantsUni.append("ඤ")
    consonants.append("KN")
    consonantsUni.append("ළු")
    consonants.append("Lu")
    consonantsUni.append("ඛ")
    consonants.append("kh")
    consonantsUni.append("ඨ")
    consonants.append("Th")
    consonantsUni.append("ඪ")
    consonants.append("Dh")
    consonantsUni.append("ශ")
    consonants.append("S")
    consonantsUni.append("ද")
    consonants.append("d")
    consonantsUni.append("ච")
    consonants.append("c")
    consonantsUni.append("ත")
    consonants.append("th")
    consonantsUni.append("ට")
    consonants.append("t")
    consonantsUni.append("ක")
    consonants.append("k")
    consonantsUni.append("ඩ")
    consonants.append("D")
    consonantsUni.append("න")
    consonants.append("n")
    consonantsUni.append("ප")
    consonants.append("p")
    consonantsUni.append("බ")
    consonants.append("b")
    consonantsUni.append("ම")
    consonants.append("m")
    consonantsUni.append("‍ය")
    consonants.append("\\u005C" + "y")
    consonantsUni.append("‍ය")
    consonants.append("Y")
    consonantsUni.append("ය")
    consonants.append("y")
    consonantsUni.append("ජ")
    consonants.append("j")
    consonantsUni.append("ල")
    consonants.append("l")
    consonantsUni.append("ව")
    consonants.append("v")
    consonantsUni.append("ව")
    consonants.append("w")
    consonantsUni.append("ස")
    consonants.append("s")
    consonantsUni.append("හ")
    consonants.append("h")
    consonantsUni.append("ණ")
    consonants.append("N")
    consonantsUni.append("ළ")
    consonants.append("L")
    consonantsUni.append("ඛ")
    consonants.append("K")
    consonantsUni.append("ඝ")
    consonants.append("G")
    consonantsUni.append("ඵ")
    consonants.append("P")
    consonantsUni.append("ඹ")
    consonants.append("B")
    consonantsUni.append("ෆ")
    consonants.append("f")
    consonantsUni.append("ග")
    consonants.append("g")
    # last because we need to ommit this in dealing with Rakaransha
    consonantsUni.append("ර")
    consonants.append("r")
    specialCharUni.append("ෲ")
    specialChar.append("ruu")
    specialCharUni.append("ෘ")
    specialChar.append("ru")
    # specialCharUni[2]="්‍ර" specialChar[2]="ra"

initializeVar()

def convertText(text):
    s=""
    r=""
    v=""
    # text = document.txtBox.box1.value;
    # special consonents
    for i in range(len(specialConsonants)):
        text = text.replace(specialConsonants[i], specialConsonantsUni[i])
# consonents + special Chars
    for i in range(len(specialCharUni)):
        for j in range(len(consonants)):
            s = consonants[j] + specialChar[i]
            v = consonantsUni[j] + specialCharUni[i]
            # r = new RegExp(s, "g")
            r = s.replace(s+"/G", "")
            text = text.replace(r, v)

# consonants + Rakaransha + vowel modifiers
    for j in range(len(consonants)):
        for i in range(len(vowels)):
            s = consonants[j] + "r" + vowels[i]
            v = consonantsUni[j] + "්‍ර" + vowelModifiersUni[i]
            # r = new RegExp(s, "g")
            r = s.replace(s+"/G", "")
            text = text.replace(r, v)
        s = consonants[j] + "r"
        v = consonantsUni[j] + "්‍ර"
        # r = new RegExp(s, "g")
        r = s.replace(s+"/G", "")
        text = text.replace(r, v)
# consonents + vowel modifiers
    for i in range(len(consonants)):
        for j in range(nVowels):
            s = consonants[i] + vowels[j]
            v = consonantsUni[i] + vowelModifiersUni[j]
            # r = new RegExp(s, "g")
            r = s.replace(s+"/G", "")
            text = text.replace(r, v)

# consonents + HAL
    for i in range(len(consonants)):
        r = consonants[i].replace(consonants[i]+"/G", "")
        text = text.replace(r, consonantsUni[i] + "්")
# vowels
    for i in range(len(vowels)):
        # r = new RegExp(vowels[i], "g")
        r = vowels[i].replace(vowels[i]+"/G", "")
        text = text.replace(r, vowelsUni[i])

    return text

#Other Functions

In [5]:
# Function to read the dictionary file and return a dictionary of Singlish to Sinhala words
def load_dictionary(file_path):
    singlish_to_sinhala = {}

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # Split the line into Singlish word and Sinhala words part
            parts = line.split(", Sinhala Words: ")
            if len(parts) == 2:
                singlish_word = parts[0].replace("Word: ", "").strip()
                sinhala_words = eval(parts[1].strip())  # Convert the string representation of list to an actual list
                singlish_to_sinhala[singlish_word] = sinhala_words
    return singlish_to_sinhala


# Function to get the Sinhala word for a given Singlish word
def get_sinhala_word(singlish_word, dictionary):
    # print(f"get_singlish wor func: {dictionary.get(singlish_word, convertText(str(singlish_word)))}")
    # print(type(dictionary.get(singlish_word, convertText(str(singlish_word)))))
    sinhala_word = dictionary.get(singlish_word, "Word not found")
    if sinhala_word == "Word not found":
      sinhala_word = [convertText(str(singlish_word))]
    return sinhala_word


# Remove words which are not in the BERT vocabulary
def clean_words(sinhala_words, tokenizer):
    new_sinhala_words = []
    vocab = set(tokenizer.vocab)  # Convert vocab to a set for O(1) lookups

    for words in sinhala_words:
        if len(words) == 1:
            new_sinhala_words.append(words)
            continue
        clean_words = [word for word in words if word in vocab]

        if not clean_words:  # If no words are found in the vocab
            clean_words = [words[0]]
        new_sinhala_words.append(clean_words)

    return new_sinhala_words


# Get the candidate words and mask the sentence accordingly
def process_sentence(new_sinhala_words):
    sinhala_words_processing = new_sinhala_words.copy()
    candidate_words = []

    # Use list comprehension and handle masking
    masked_sentence = []
    for words in sinhala_words_processing:
        if len(words) > 1:
            candidate_words.append(words)
            masked_sentence.append('[MASK]')
        else:
            masked_sentence.append(' '.join(words))

    # Use 'join' to build the sentence efficiently
    masked_sentence = ' '.join(masked_sentence)
    return masked_sentence, candidate_words


# Split the input sentence and get the relevant native Sinhala words
def get_sinhala_words(singlish_sentence, singlish_to_sinhala_dict):
    # Separate words by spaces and get corresponding Sinhala words
    singlish_words = singlish_sentence.split()
    sinhala_words = [get_sinhala_word(word, singlish_to_sinhala_dict) for word in singlish_words]
    return sinhala_words


# generate the sentence combinations with candidate words
def generate_sentences_with_all_combinations(template, candidates):
    # Split the template by spaces
    template_parts = template.split()

    # Identify the indices of blanks
    blank_indices = [i for i, word in enumerate(template_parts) if word == "[MASK]"]

    # Generate all combinations of candidate words for blanks
    all_combinations = list(itertools.product(*candidates))

    # Generate all full sentences
    all_sentences = []
    for combo in all_combinations:
        filled_sentence = template_parts.copy()  # Start with the template
        for i, word in zip(blank_indices, combo):  # Replace blanks with the combo words
            filled_sentence[i] = word
        all_sentences.append(" ".join(filled_sentence))  # Join words into a sentence

    return all_sentences


def fill_blanks(blanked_sentence, combination):
  # Split the sentence by the '[BLANK]' placeholder
  parts = blanked_sentence.split('[MASK]')
  # Rebuild the sentence by joining the parts and filling in the blanks
  filled_sentence = ''.join(f"{part}{word}" for part, word in zip(parts, combination + ('',)))
  return filled_sentence


def generate_probs(sentence_with_blank, candidates, tokenizer):
  # Tokenize the sentence
  input_ids = tokenizer.encode(sentence_with_blank, return_tensors='pt')

  # Find the position of the mask token
  mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1].item()

  # Get the logits (unnormalized probabilities) for each token in the sentence
  with torch.no_grad():
      outputs = model(input_ids)
      logits = outputs.logits

  # Get the logits for the mask token position
  mask_token_logits = logits[0, mask_token_index, :]

  # Get the probabilities for the candidate words
  word_ids = tokenizer.convert_tokens_to_ids(candidates)
  word_probs = F.softmax(mask_token_logits, dim=-1)[word_ids].tolist()

  zipped = zip(candidates, word_probs)
  return zipped


# generate the sentence combinations with candidate words
def generate_sentences_with_all_combinations(template, candidates):
    # Split the template by spaces
    template_parts = template.split()

    # Identify the indices of blanks
    blank_indices = [i for i, word in enumerate(template_parts) if word == "[MASK]"]

    # Generate all combinations of candidate words for blanks
    all_combinations = list(itertools.product(*candidates))

    # Generate all full sentences
    all_sentences = []
    for combo in all_combinations:
        filled_sentence = template_parts.copy()  # Start with the template
        for i, word in zip(blank_indices, combo):  # Replace blanks with the combo words
            filled_sentence[i] = word
        all_sentences.append(" ".join(filled_sentence))  # Join words into a sentence

    return all_sentences


# Function to calculate the product of values for a given sentence
def calculate_product(sentence, word_probabilities):
    product = 1
    found = False
    for key, value in word_probabilities.items():
        parts = key.split("--")

        if re.sub(r'\s+', ' ', parts[2]).strip() == sentence:
            product *= value
            found = True
    return product if found else None


# Generate sentences with one blank
def generate_sentences_with_one_blank(word_combinations, mask_indexes, masked_sentence):
    one_blank_sentences = {}
    for comb in word_combinations:
      for index, mask_index in enumerate(mask_indexes):
        combination = list(comb)
        masking_word = combination[index]
        combination[index] = '[MASK]'
        one_blank_sentence = fill_blanks(masked_sentence, tuple(combination))

        if one_blank_sentence in one_blank_sentences.keys():
          one_blank_sentences[one_blank_sentence].append(masking_word)
        else:
          one_blank_sentences[one_blank_sentence] = [masking_word]

    return one_blank_sentences


# Generate probability dictinary
def generate_probability_dict(one_blank_sentences, tokenizer):
    word_probabilities = {}
    for one_blank_sentence, candidate in one_blank_sentences.items():
      probs = list(generate_probs(one_blank_sentence, candidate, tokenizer))

      for i in range(len(probs)):
        word = probs[i][0]
        full_sentence = one_blank_sentence.replace('[MASK]', word)
        sentence_key = one_blank_sentence + "--" + word + "--" + full_sentence
        prob = probs[i][1]
        word_probabilities[sentence_key] = prob

    return word_probabilities



def calculate_number_of_bert_calls(candidate_words):
    n = len(candidate_words)
    # If there's only one blank, there is only one possible sentence with that blank remaining
    if n == 1:
        return 1

    total_sentences = 0

    # Iterate over each blank
    for i in range(n):
        # Calculate the product of candidate words for all other blanks
        product = reduce(lambda x, y: x * y,
                         (len(candidate_words[j]) for j in range(n) if j != i))
        total_sentences += product

    return total_sentences


def get_key_by_value(dictionary, value):
    """
    This function searches a dictionary for a given value and returns the corresponding key.
    If the value is not found, it returns None.
    """
    for key, val in dictionary.items():
        if val == value:
            return key
    return None


def numbering_masks_sentences(sentences):
    # Initialize the mask counter and a dictionary to keep track of mappings
    mask_counter = 1
    mask_map = {}
    previous_masks = []

    updated_sentences = []

    for i, sentence in enumerate(sentences):
        parts = sentence.split("[MASK]")
        labeled_sentence = parts[0]

        for j in range(1, len(parts)):
            # For the first two masks in a sentence (if not the first sentence), use previous masks
            if i > 0 and j <= 2:
                label = previous_masks[j - 1]
            else:
                # Otherwise, assign a new label if not already assigned
                mask_key = (i, j)
                if mask_key in mask_map:
                    label = mask_map[mask_key]
                else:
                    label = f"MASK{mask_counter}"
                    mask_map[mask_key] = label
                    mask_counter += 1

            labeled_sentence += f"[{label}]" + parts[j]

            # Save the last two labels to use in the next sentence for overlap
            if j >= len(parts) - 2:
                previous_masks.append(label)

        # Only retain the last two labels for overlap with the next sentence
        previous_masks = previous_masks[-2:]

        updated_sentences.append(labeled_sentence)

    return updated_sentences


# Replace [MASK] with numbered labels
def numbering_masks_sentence(sentence):
    masks = re.findall(r'\[MASK\]', sentence)
    for i, _ in enumerate(masks, 1):
        sentence = sentence.replace("[MASK]", f"[MASK{i}]", 1)
    return sentence


# Function to extract words for each mask
def find_mask_words(mask_sentences, labeled_sentences):
    mask_words = defaultdict(list)

    # Iterate over sentences with masks and corresponding labeled sentences
    for mask_sentence, labeled_sentence in zip(mask_sentences, labeled_sentences):
        mask_parts = re.findall(r'\[MASK\d+\]', mask_sentence)
        labeled_parts = labeled_sentence.split()

        # Iterate over mask parts and map them to corresponding words
        mask_index = 0
        for i, part in enumerate(mask_sentence.split()):
            if mask_index < len(mask_parts) and part == mask_parts[mask_index]:
                word = labeled_parts[i]
                mask_words[mask_parts[mask_index]].append(word)
                mask_index += 1

    # Remove duplicates and format the output
    mask_words = {mask: list(set(words)) for mask, words in mask_words.items()}
    return mask_words


# Function to replace MASKs and collect candidates
def replace_masks_and_collect_candidates(sentence, mask_words):
    candidates = []

    for mask, words in mask_words.items():
        if len(words) > 1:
            # Replace with [MASK] if there are multiple words
            sentence = sentence.replace(f"[{mask[1:-1]}]", "[MASK]")
            candidates.append(words)
        else:
            # Replace with the single word
            sentence = sentence.replace(f"[{mask[1:-1]}]", words[0])

    return sentence, candidates


def chunk_sentence(mask_sentence, candidate_words):
  mask_sentence = mask_sentence.split()
  mask_indexes = [index for index, word in enumerate(mask_sentence) if word == '[MASK]']

  # Map candidate_words indexes with mask_indexes
  index_map = {}
  for i in range(len(mask_indexes)):
    index_map[i] = mask_indexes[i]

  max_bert_call = 20
  overlap = 2
  mask_count = 0
  sentences = []
  candidates = []
  sub_sentence = []
  sub_candidates = []


  for index, word in enumerate(mask_sentence):
    if word != "[MASK]":
      sub_sentence.append(word)
      if index == len(mask_sentence)-1:
        sentences.append(" ".join(sub_sentence))
        candidates.append(sub_candidates)
    else:
      sub_candidates.append(candidate_words[get_key_by_value(index_map, index)])
      sub_sentence.append(word)
      mask_count += 1

      if (len(sub_candidates)>= (overlap+1)):

        if (calculate_number_of_bert_calls(sub_candidates)) > max_bert_call:
          try:
            sub_sentence = sub_sentence + mask_sentence[mask_indexes[mask_count-1]+1:mask_indexes[mask_count]]
          except:
            sub_sentence = sub_sentence + mask_sentence[mask_indexes[mask_count-1]+1:]

          sentences.append(" ".join(sub_sentence))
          candidates.append(sub_candidates)

          sub_sentence = mask_sentence[(mask_indexes[(mask_count-overlap)-1])+1:index+1]
          sub_candidates = sub_candidates[-2:]

        elif index == len(mask_sentence)-1:
          sentences.append(" ".join(sub_sentence))
          candidates.append(sub_candidates)

  return sentences, candidates


# transliterating process
def transliterate(masked_sentence, candidates, tokenizer):

  word_combinations = list(itertools.product(*candidates))
  word_list = masked_sentence.split()
  mask_indexes = [index for index, word in enumerate(word_list) if word == '[MASK]']

  # generate sentences with one blanks including possible candidae words for the blank
  one_blank_sentences = generate_sentences_with_one_blank(word_combinations, mask_indexes, masked_sentence) #dictionary

  word_probabilities = generate_probability_dict(one_blank_sentences, tokenizer)
  full_sentences = generate_sentences_with_all_combinations(masked_sentence, candidates)

  # Find the sentence with the highest product
  max_product = None
  max_sentence = None

  for sentence in full_sentences:
      product = calculate_product(sentence, word_probabilities)
      if product is not None and (max_product is None or product > max_product):
          max_product = product
          max_sentence = sentence

  # print(f"Output: {max_sentence}")
  return max_sentence


def generate_sinhala(singlish_sentence, singlish_to_sinhala_dict, tokenizer):
  sinhala_words = get_sinhala_words(singlish_sentence, singlish_to_sinhala_dict)
  filtered_sinhala_words = clean_words(sinhala_words, tokenizer)
  # print(f"Filtered words: {filtered_sinhala_words}\n")
  masked_sentence, candidates = process_sentence(filtered_sinhala_words)

  while True:
    if len(candidates) == 0:
      # print(f"Output: {masked_sentence}")
      return masked_sentence
      # return masked_sentence, ["No candidates"]

    else:
      # print(f"Masked sentence: {masked_sentence}")
      # print(f"Candidate words: {candidates}\n")

      if len(candidates) <= 3:
        output = transliterate(masked_sentence, candidates, tokenizer)
        # print(f"Output: {output}")
        return output
      else:
        sentences, candidates = chunk_sentence(masked_sentence, candidates)
        # print(f"CHUNKED Sentences: {sentences}")
        # print(f"CHUNKED Candidates: {candidates}\n")

        # Numbering masks
        numbered_input_sentence = numbering_masks_sentence(masked_sentence)
        # print(f"Numbered input sentence: {numbered_input_sentence}")
        numbered_sentences = numbering_masks_sentences(sentences)
        # print(f"Numbered sentences: {numbered_sentences}")

        filled_sentences = [transliterate(sentences[i], candidates[i], tokenizer) for i in range(len(sentences))]
        # print(f"Filled sentences: {filled_sentences}")

        # Find the words for each mask
        mask_words = find_mask_words(numbered_sentences, filled_sentences)
        # print(f"Mask words: {mask_words}")

        # Replace the MASKs and collect candidates
        masked_sentence, candidates = replace_masks_and_collect_candidates(numbered_input_sentence, mask_words)
        # print(f"Updated sentence: {masked_sentence}")
        # print(f"Updated Candidates: {candidates}\n")

#Model & Tokenizer

In [7]:
model_name = "Sameera827/sinhala-bert-dakshina_new01"
model_directory = "/content/drive/MyDrive/Singlish-Back-Transliterator/Model"

tokenizer = AutoTokenizer.from_pretrained("Ransaka/sinhala-bert-medium-v2")
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Load the tokenizer and model from the local directory
# tokenizer = AutoTokenizer.from_pretrained(model_directory)
# model = AutoModelForMaskedLM.from_pretrained(model_directory)

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/485k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/988k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/202M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


#Load Singlish-Sinhala Dictionary

In [8]:
# Load Dictionary
file_path = "/content/drive/MyDrive/Singlish-Transliterator/SinglishDict/SinglishDic/WSD Romanized-Sinhala - Sinhala .txt"  # Replace with the actual path to your txt file
singlish_to_sinhala_dict = load_dictionary(file_path)

#Tranliteration

In [12]:
singlish_sentence = input("Enter Singlish Sentence: ")
singlish_sentence = re.sub(r'\s+', ' ', singlish_sentence).strip()
singlish_sentence

Enter Singlish Sentence: mm ad gedr ymi


'mm ad gedr ymi'

In [13]:
# singlish_sentence = "mama hithuwe oya neethi reethi anuwa kreeda kala kiyala mama hithuwe oya ehema kale nehe kiyala"
# singlish_sentence = re.sub(r'\s+', ' ', singlish_sentence).strip()
# singlish_sentence

In [14]:
%%time
generate_sinhala(singlish_sentence, singlish_to_sinhala_dict, tokenizer)

CPU times: user 792 ms, sys: 6.77 ms, total: 798 ms
Wall time: 800 ms


'මම අද ගෙදර යමි'

#Test Data Seperation

In [ ]:
# # Define file paths
# input_file_path = '/content/drive/MyDrive/Singlish-Back-Transliterator/Test_Data/Original/Sinhala Test set 2.txt'
# output_directory = '/content/drive/MyDrive/Singlish-Back-Transliterator/Test_Data/Separated/Test_Set_2'  # Ensure this directory exists

# # Define the maximum number of sentence pairs per file
# max_pairs_per_file = 3000  # Adjust this as needed

# # Initialize counters
# pair_count = 0
# file_count = 1

# # Open the first output file
# output_file = open(f'{output_directory}/output_part2_{file_count}.txt', 'w', encoding='utf-8')

# with open(input_file_path, 'r', encoding='utf-8') as infile:
#     # Loop through each line, storing pairs
#     for line in infile:
#         # Strip any leading/trailing whitespace
#         line = line.strip()

#         # Check if line is not empty
#         if line:
#             # Write the line to the current output file
#             output_file.write(line + '\n')
#             pair_count += 1

#             # After writing a pair (two lines), add a blank line
#             if pair_count % 2 == 0:
#                 output_file.write('\n')  # Add a blank line between pairs

#                 # Check if we have reached the max pairs limit for the current file
#                 if pair_count // 2 >= max_pairs_per_file:
#                     output_file.close()
#                     file_count += 1
#                     pair_count = 0
#                     output_file = open(f'{output_directory}/output_part2_{file_count}.txt', 'w', encoding='utf-8')

# # Close the last output file
# output_file.close()


#Generate Results for Test Data

In [ ]:
# %%time

# def classify_line(line):
#     """Classify the line as Romanized, Sinhala, or blank."""
#     line = line.strip()

#     # Check if the line is blank
#     if not line:
#         return "Blank"

#     # Check if the line contains Sinhala characters (Unicode range 0D80–0DFF)
#     if any('\u0D80' <= char <= '\u0DFF' for char in line):
#         return "Sinhala"

#     # If it's not blank and doesn't contain Sinhala characters, it's Romanized
#     return "Romanized"

# # Open the input file and read the content
# with open('/content/drive/MyDrive/Singlish-Back-Transliterator/Test_Data/Original/Sinhala Test set 2.txt', 'r', encoding='utf-8') as file:
#     # Read the lines from the file
#     lines = file.readlines()

# # Open a new file to write the filtered pairs
# with open('/content/drive/MyDrive/Singlish-Back-Transliterator/Test_Data/Results/Test_Data_2/Test_Data_Result_2.txt', 'w', encoding='utf-8') as output_file:

#     # Variables to hold Romanized and Sinhala pairs
#     romanized_sentence = None
#     sinhala_sentence = None

#     # Loop through the lines and classify them
#     for i, line in enumerate(lines, start=1):
#         classification = classify_line(line)

#         if classification == "Romanized":
#             romanized_sentence = line.strip()  # Store the Romanized line
#         elif classification == "Sinhala":
#             sinhala_sentence = line.strip()  # Store the Sinhala line

#         # If both Romanized and Sinhala sentences are identified
#         if romanized_sentence and sinhala_sentence:
#             # Count the number of words in the Romanized sentence
#             word_count = len(romanized_sentence.split())

#             output_file.write(f"Romanized Input: {romanized_sentence}\n")
#             output_file.write(f"Expected Sinhala: {sinhala_sentence}\n")


#             time_start = time.time()
#             output = generate_sinhala(re.sub(r'\s+', ' ', romanized_sentence).strip(), singlish_to_sinhala_dict, tokenizer)
#             time_end = time.time()

#             time_taken = time_end - time_start

#             output_file.write(f"Generated Sinhala: {output}\n")
#             output_file.write(f"Time taken: {time_taken} seconds\n")

#             print(f"Romanized Input: {romanized_sentence}")
#             print(f"Expected Sinhala: {sinhala_sentence}")
#             print(f"Generated Sinhala: {output}\n")

#             output_file.write("\n")  # Write a blank line for readability

#             # Reset the variables after processing the pair
#             romanized_sentence = None
#             sinhala_sentence = None


Streaming output truncated to the last 5000 lines.
Generated Sinhala: ඔබ පූර්ණ කාලීනව ආයතනයේ සිටිනවාද නැතිනම් ඔබට රැකියාවක් තිබේද එය රැකියාවක් මම සමහර ආධුනිකයෙකු මෙන් රැවටෙන්නෙ නැත එය මගේ ජීවිතයයි

Romanized Input: mma smhra adhunikayek wge rewtenne nethi rekiywk meka mge jeewithya apita ikmninma apahu enna wenwa
Expected Sinhala: මම සමහර ආධුනිකයෙක් වගේ රැවටෙන්නේ නැති රැකියාවක් මේක මගේ ජීවිතය අපිට ඉක්මනින්ම ආපහු එන්න වෙනවා
Generated Sinhala: මම සමහර ආධුනිකයෙක් වගේ රැවටෙන්නෙ නැති රැකියාවක් මේක මගේ ජීවිතය අපිට ඉක්මනින්ම ආපහු එන්න වෙනවා

Romanized Input: mokda wune kiyla mta blnna ona mge depa allagena inna deyk
Expected Sinhala: මොකද වුණේ කියලා මට බලන්න ඕන මගේ දෙපා අල්ලාගෙන ඉන්න දෙයක්
Generated Sinhala: මොකද වුනේ කියල මට බලන්න ඕන මගේ දෙපා අල්ලගෙන ඉන්න දෙයක්

Romanized Input: owun sthuwa jeewithrkshka jekt ho amathara bottuwk nomethi nm owun ym deykta phra diya yuthuya
Expected Sinhala: ඔවුන් සතුව ජීවිතාරක්ෂක ජැකට් හෝ අමතර බෝට්ටුවක් නොමැති නම් ඔවුන් යම් දෙයකට පහර දිය යුතුය
Generated Sinha

#Evaluation

***Evaluation: Test Set 1***

In [ ]:
# Define file path
file_path = '/content/drive/MyDrive/Singlish-Back-Transliterator/Test_Data/Results/Test_Data_1/Test_Data_Results_1.txt'

# Initialize lists to store expected and generated Sinhala sentences
ground_truth = []
generated_output = []

# Read and parse the file
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Check if the line contains 'Expected Sinhala' or 'Generated Sinhala'
        if line.startswith("Expected Sinhala:"):
            # Extract the expected Sinhala sentence
            expected_sentence = line.split("Expected Sinhala:")[1].strip()
            ground_truth.append(expected_sentence)
        elif line.startswith("Generated Sinhala:"):
            # Extract the generated Sinhala sentence
            generated_sentence = line.split("Generated Sinhala:")[1].strip()
            generated_output.append(generated_sentence)



# WER and CER
word_error_rate = wer(ground_truth, generated_output)
character_error_rate = cer(ground_truth, generated_output)
print(f"WER: {word_error_rate}")
print(f"CER: {character_error_rate}\n")

#BLEU
ground_truth = [[ref.split()] for ref in ground_truth]  # Wrap each reference in a list for corpus_bleu
generated_output = [hyp.split() for hyp in generated_output]  # Tokenize each hypothesis

# Calculate BLEU score
bleu_score1 = corpus_bleu(ground_truth, generated_output, weights=(1.0, 0, 0, 0))
bleu_score2 = corpus_bleu(ground_truth, generated_output, weights=(0.5, 0.5, 0, 0))
bleu_score3 = corpus_bleu(ground_truth, generated_output, weights=(0.3, 0.3, 0.3, 0))
bleu_score4 = corpus_bleu(ground_truth, generated_output, weights=(0.25, 0.25, 0.25, 0.25))


print(f"BLEU-1: {bleu_score1}")
print(f"BLEU-2: {bleu_score2}")
print(f"BLEU-3: {bleu_score3}")
print(f"BLEU-4: {bleu_score4}")


WER: 0.08861589936199177
CER: 0.020034772485313605

BLEU-1: 0.911468284882529
BLEU-2: 0.8718182943031035
BLEU-3: 0.8487933998536085
BLEU-4: 0.79632886827261


***Evaluation: Test Set 2***

In [ ]:
# Define file path
file_path = '/content/drive/MyDrive/Singlish-Back-Transliterator/Test_Data/Results/Test_Data_2/Test_Data_Result_2.txt'

# Initialize lists to store expected and generated Sinhala sentences
ground_truth = []
generated_output = []

# Read and parse the file
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Check if the line contains 'Expected Sinhala' or 'Generated Sinhala'
        if line.startswith("Expected Sinhala:"):
            # Extract the expected Sinhala sentence
            expected_sentence = line.split("Expected Sinhala:")[1].strip()
            ground_truth.append(expected_sentence)
        elif line.startswith("Generated Sinhala:"):
            # Extract the generated Sinhala sentence
            generated_sentence = line.split("Generated Sinhala:")[1].strip()
            generated_output.append(generated_sentence)



# WER and CER
word_error_rate = wer(ground_truth, generated_output)
character_error_rate = cer(ground_truth, generated_output)
print(f"WER: {word_error_rate}")
print(f"CER: {character_error_rate}\n")

#BLEU
ground_truth = [[ref.split()] for ref in ground_truth]
generated_output = [hyp.split() for hyp in generated_output]

# Calculate BLEU score
bleu_score1 = corpus_bleu(ground_truth, generated_output, weights=(1.0, 0, 0, 0))
bleu_score2 = corpus_bleu(ground_truth, generated_output, weights=(0.5, 0.5, 0, 0))
bleu_score3 = corpus_bleu(ground_truth, generated_output, weights=(0.3, 0.3, 0.3, 0))
bleu_score4 = corpus_bleu(ground_truth, generated_output, weights=(0.25, 0.25, 0.25, 0.25))


print(f"BLEU-1: {bleu_score1}")
print(f"BLEU-2: {bleu_score2}")
print(f"BLEU-3: {bleu_score3}")
print(f"BLEU-4: {bleu_score4}")


WER: 0.09139198018672706
CER: 0.021238127233301773

BLEU-1: 0.9087561508130478
BLEU-2: 0.868619537170309
BLEU-3: 0.8452052560776309
BLEU-4: 0.7917129259807374
